# 教材の更新確認
---

ホームディレクトリと `textbook` フォルダの教材を比較して、変更された教材ファイルの一覧を表示します。

次のセルを実行して教材の更新の有無を確認してください。

In [ ]:
import os, sys, urllib.parse, IPython.display, subprocess

course_name = os.environ['CWH_COURSE_NAME']
server_name = os.environ['JUPYTERHUB_SERVER_NAME']

assert course_name != '', "コース用のサーバー上で実行してください"

HOME_DIR = os.path.expanduser('~')
TEXTBOOK_DIR = os.path.join(HOME_DIR, course_name, 'textbook')

def get_released_files():
    for root, dirs, files in os.walk(TEXTBOOK_DIR):
        for file in files:
            yield os.path.relpath(os.path.join(root, file), start=TEXTBOOK_DIR)

rows = []
for file in sorted(get_released_files()):
    my_file = os.path.join(HOME_DIR, course_name, file)
    if os.path.isfile(my_file):
        try:
            cmd = './notebook-diff {} {}'.format(os.path.join(TEXTBOOK_DIR, file), my_file)
            diff = subprocess.run(cmd, shell=True)
            if diff.returncode == 0:
                rows.append((0, file, None))
            else:
                u = None
                if file.endswith('.ipynb'):
                    p1 = urllib.parse.quote(file, safe='')
                    p2 = urllib.parse.quote('/'.join(('textbook', file)), safe='')
                    u = '/user-redirect/{}/tree?diffpath1={}&amp;diffpath2={}#notebook_diff'.format(server_name, p2, p1)
                rows.append((1, file, u))
        except OSError as e:
            print('ERROR: notebook-diff failed: {}'.format(e), file=sys.stderr)
    else:
        rows.append((2, file, None))

html = [
    '<h3>',
    '更新があります' if [r for r in rows if r[0]] else '更新はありません',
    '</h3>',
    '<table>'
]
for row in rows:
    html.extend([
        '<tr style="color:{};">'.format(['#999', '#66c', '#c66'][row[0]]),
        '<td style="text-align:center;">',
        ['変更なし', '変更あり', '新規'][row[0]],
        '<td style="text-align:left;">',
        '<code style="color:inherit;background-color:transparent;">',
        row[1],
        '</code>',
        '<td>'
    ])
    if row[2] is not None:
        html.append('<a href="{}" target="nbdiff">差分表示</a>'.format(row[2]))
html.append('</table>')

IPython.display.display(IPython.display.HTML(''.join(html)))

---
# 教材の更新

上のセルを実行した結果、「変更あり」もしくは「新規」の教材を取得して更新します。
対象となる Notebook に対して行なっている編集や、実行結果は、全て上書きされ破棄されますのでご注意ください。

次のセルを実行すると、教材が更新されます。

In [ ]:
import os, sys, os.path, shutil

if not 'rows' in globals():
    print('上のセルを実行してください。', file=sys.stderr)
else:
    updated = False
    for row in [r for r in rows if r[0]]:
        src = os.path.join(TEXTBOOK_DIR, row[1])
        dst = os.path.join(HOME_DIR, course_name, row[1])
        dstdir = os.path.dirname(dst)
        if not os.path.isdir(dstdir):
            os.makedirs(dstdir, mode=0o755, exist_ok=True)
        shutil.copy2(src, dst)
        print('{} -> {}'.format(src,dst))
        if dst.endswith('.ipynb'):
            try:
                cmd = 'jupyter-trust {}'.format(dst)
                proc = subprocess.run(cmd, shell=True)
                if proc.returncode != 0:
                    print('trust に失敗: {}'.format(dst), file=sys.stderr)
                updated = True
            except OSError as e:
                print('ERROR: jupyter-trust failed: {}'.format(e), file=sys.stderr)
    if updated:
        print('教材を更新しました。')
    else:
        print('更新はありません。', file=sys.stderr)